Preprocess data

In [1]:
import pandas as pd
from tqdm import tqdm
import csv

In [2]:
brandbankExtract = r'Data\Bulk Extract.csv'

In [3]:
df = pd.read_csv(brandbankExtract)
df

,name,gtin,isDelete,date,pl1,pl2,pl3,description,allergyAdvice,recycling,recycling_other,brands,marketing,ingredients,features,storage,preserves
0,905597.xml,185,0,2010-08-16,Dairy & Bread,White Bread & Rolls,Other,Waitrose Fig & Raisin Loaf 400g,NaN,NaN,NaN,Waitrose,NaN,NaN,NaN,NaN,NaN
1,643362.xml,246,0,2006-02-24,Deli,Unallocated,Unallocated,WR Gingerbread biscuit chick each,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,905600.xml,260,0,2010-08-16,Dairy & Bread,White Bread & Rolls,French Bread/ Rolls,Waitrose Petit Parisienne 290g,NaN,NaN,NaN,Waitrose,NaN,NaN,NaN,NaN,NaN
3,980223.xml,581,0,2011-02-28,Fruit & Vegetables,Salad Products,Peppers,Waitrose Loose Yellow Peppers,NaN,NaN,NaN,Waitrose,NaN,NaN,NaN,NaN,NaN
4,8445550.xml,6491,0,2020-10-14,Fruit & Vegetables,Apples,Other Apples,Asda Pink Lady Apples,NaN,NaN,NaN,Asda,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157840,2375232.xml,9857423623122,0,2014-08-12,Meat & Fish,Fresh Fish,Salmon,Wrights of Howth Sliced Smoked Salmon 400gr,NaN,NaN,NaN,Wrights of Howth,NaN,"Salmon, Salmo Salar Salt, Oak Smoke",NaN,Keep Refrigerated,NaN
157841,5596613.xml,9993510358956,0,2018-03-19,Meat & Fish,Shellfish,Other Shellfish,Morrisons Fresh Live Scottish Mussels 1kg,NaN,NaN,NaN,Morrisons,NaN,NaN,NaN,"Not Suitable for Home Freezing, Keep Refrigerated",NaN
157842,4607178.xml,9999999999994,0,2017-07-20,Meat & Fish,Lamb,Other Lamb,Glenmór Lamb Carvery Shoulders,NaN,NaN,NaN,Glenmór,NaN,NaN,Irish country meats,NaN,NaN
157843,6771345.xml,9999999999994,0,2019-03-06,Meat & Fish,Reformed Meats,Burgers,Hogan's Farm 12 Buffalo Turkey Burgers and 12 ...,"Sulphur Dioxide/Sulphites, Contains",NaN,NaN,Hogan's Farm,NaN,"Turkey (75%), Buffalo Glaze (Acidity Regulator...","High in protein, Low in fat, Fresh class A tur...",NaN,NaN


In [4]:
df=df.fillna('')
df = df[df['isDelete'] != 1]
df['text'] = df[[
    'description',
    'allergyAdvice',
    'recycling',
    'recycling_other',
    'brands',
    'marketing',
    'ingredients',
    'features',
    'storage',
    'preserves']].agg(' '.join, axis=1)


df['text'] = df['text'].str.lower()
df.head()

,name,gtin,isDelete,date,pl1,pl2,pl3,description,allergyAdvice,recycling,recycling_other,brands,marketing,ingredients,features,storage,preserves,text
0,905597.xml,185,0,2010-08-16,Dairy & Bread,White Bread & Rolls,Other,Waitrose Fig & Raisin Loaf 400g,,,,Waitrose,,,,,,waitrose fig & raisin loaf 400g waitrose
1,643362.xml,246,0,2006-02-24,Deli,Unallocated,Unallocated,WR Gingerbread biscuit chick each,,,,,,,,,,wr gingerbread biscuit chick each
2,905600.xml,260,0,2010-08-16,Dairy & Bread,White Bread & Rolls,French Bread/ Rolls,Waitrose Petit Parisienne 290g,,,,Waitrose,,,,,,waitrose petit parisienne 290g waitrose
3,980223.xml,581,0,2011-02-28,Fruit & Vegetables,Salad Products,Peppers,Waitrose Loose Yellow Peppers,,,,Waitrose,,,,,,waitrose loose yellow peppers waitrose
4,8445550.xml,6491,0,2020-10-14,Fruit & Vegetables,Apples,Other Apples,Asda Pink Lady Apples,,,,Asda,,,,,,asda pink lady apples asda


In [7]:
df.to_csv(r'Data\Bulk Extract Text.csv',index=False)

In [36]:
import psycopg2
#Establishing the connection
conn = psycopg2.connect(
   database="Products_Phrases", user='Jack Simpson@impactscoredb', password='hycMK>~4~3', host='impactscoredb.postgres.database.azure.com', port= '5432'
)
cursor = conn.cursor()

In [ ]:
for index, row in df.iterrows():
    cursor.execute('''INSERT INTO "brandbankxml" VALUES (%s, %s, %s,%s, %s,%s, %s, %s,%s, %s,%s, %s, %s,%s, %s,%s, %s, %s)''',(row['name'],row['gtin'],row['isDelete'],row['date'],row['pl1'],row['pl2'],row['pl3'],row['description'],row['allergyAdvice'],row['recycling'],row['recycling_other'],row['brands'],row['marketing'],row['ingredients'],row['features'],row['storage'],row['preserves'],row['text'],))
    # Commit changes in the database
    conn.commit()

In [34]:
df_phrases = pd.read_csv(r'Extracted Phrases\Rake NLTK\Phrases Only.csv')
df_phrases

,PhraseID,Phrase
0,1,e
1,2,r
2,3,n
3,4,l
4,5,c
...,...,...
159696,159697,"triphosphates ), potassium chloride"
159697,159698,ecklonia cava extract
159698,159699,"top ), gluten free"
159699,159700,"1 %), skimmed cows"


In [38]:
for index, row in df_phrases.iterrows():
    cursor.execute('''INSERT INTO "Phrases" VALUES (%s, %s)''',(row['PhraseID'],row['Phrase']))
    # Commit changes in the database
    conn.commit()

KeyboardInterrupt: 

RAKE_NLTK

In [19]:
from rake_nltk import Rake
import nltk
r = Rake()

In [20]:
text = df['text'].to_list()

In [23]:
phrasesSet=set()
for textItem in text:
    r.extract_keywords_from_sentences([textItem])
    phrases=r.get_ranked_phrases()
    phrasesSet.update(phrases)
phrasesSet

{'beco 270 super strong degradable poop bags beco',
 '386kj',
 'ci 77007 enhances',
 'sugar per 100g',
 'bacon bbq melt',
 'miso every miso paste',
 'meringue crumbs strawberry cookie dough strawberry ice cream strawberry ice cream',
 'dipotassium phosphate ), black pepper extract',
 'pwl 7',
 'partners fairtrade berry meringue chocolate round 90g eggs',
 '1 roll ** ** based',
 'nivea fresh clean 0',
 'elegant white wine',
 'wired interconnection via ethernet',
 'static decantation prior',
 'recyclable soulful',
 'thiamine ), sultanas',
 'milk ), potato starch ], apricots',
 'moisturises skin adding',
 'tree nut allergy',
 'attractive floral aromas',
 'images',
 '1 cartridge',
 'apple juice ), gelling agent',
 'ph regulator e330',
 'small french oak barrels',
 'independent tests',
 'waitrose potato',
 'dark chocolate nougat 100g almonds',
 '200 days per year',
 'nescafé dolce gusto espresso decaffeinated',
 'pampers pure protection size 3',
 'finished product inspired',
 'recyclable ma

In [24]:
with open(r'Extracted Phrases\Rake NLTK\Individual Extraction.csv', 'w', encoding="utf-8",newline='') as f:
    writer = csv.writer(f)
    for phrase in list(phrasesSet):
        writer.writerow([phrase])

In [24]:
def get_num_words(row):
    return len(str(row['Phrase']).split())

def get_perc_phrase(row,total):
    return (float(row['Score'])/total*100)

In [25]:
def get_Rake_phrases(df):
    r = Rake()
    r.extract_keywords_from_sentences(df['text'])
    phrases=r.get_ranked_phrases_with_scores()
    df = pd.DataFrame(phrases, columns = ['Rank', 'Phrase'])
    df['Phrase'] = df['Phrase'].str.strip()
    phrase_counts = df['Phrase'].value_counts()
    phrase_counts = phrase_counts.reset_index()
    phrase_counts.columns=['Phrase','Score']
    phrase_counts['Length'] = phrase_counts.apply (lambda row: get_num_words(row), axis=1)
    phrase_counts=phrase_counts.loc[phrase_counts['Length'] <=6]
    total = float(phrase_counts['Score'].sum())
    phrase_counts['Perc'] = phrase_counts.apply (lambda row: get_perc_phrase(row,total), axis=1)
    return phrase_counts

In [26]:
df_raked = get_Rake_phrases(df)
#phrase_counts.to_csv(r'C:\Users\Clamfighter\Documents\GitHub\NLP\Impact Score Key Phrase\Extracted Phrases\Rake NLTK\Brandbank Latest Phrase Extraction.csv',index = False)

In [27]:
df_raked

,Phrase,Score,Length,Perc
0,contains,46195,1,1.027913
1,free,28158,1,0.626561
2,may contain,27941,2,0.621732
3,suitable,27103,1,0.603085
4,salt,25606,1,0.569774
...,...,...,...,...
559954,something big 8,1,3,0.000022
559955,refreshingly remarkable treat,1,3,0.000022
559956,naturally fresh deo crystal roll,1,5,0.000022
559957,"salt ), vinegar",1,3,0.000022


In [10]:
phrase_count=[]

df_phraseCount = df_raked.loc[df_raked['Score'] != 1] #filters Score of 1 out. will be added back in later
phrases = df_phraseCount['Phrase']
text = df['text']
for phrase in tqdm(phrases): #find number of phrases for all strings
    phrase_count.append(sum(phrase in s for s in text))
phrase_count

 10%|▉         | 5088/51606 [01:27<12:53, 60.16it/s]

In [ ]:
df_phraseCount['Count'] = phrase_count
df_phraseCount.drop(['Score','Length','Perc'],axis=1, inplace=True)
df_out = df_raked.merge(df_phraseCount, on='Phrase', how='left')
df_out.fillna(1,inplace=True)
df_out

C:\Users\admin_1878\AppData\Local\Temp\ipykernel_11008\2573982755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_phraseCount['Count'] = phrase_count
C:\Users\admin_1878\AppData\Local\Temp\ipykernel_11008\2573982755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_phraseCount.drop(['Score','Length','Perc'],axis=1, inplace=True)


,Phrase,Score,Length,Perc,Count
0,contains,46090,1,1.043119,46566.0
1,free,27964,1,0.632888,28091.0
2,may contain,27856,2,0.630443,14558.0
3,suitable,26888,1,0.608535,32327.0
4,sugar,25511,1,0.577371,33710.0
...,...,...,...,...,...
500458,best fighting units,1,3,0.000023,1.0
500459,piers morgan unbranded piers morgan,1,5,0.000023,1.0
500460,extremely perilous christmas,1,3,0.000023,1.0
500461,trifles unbranded,1,2,0.000023,1.0


In [ ]:
#df_out.to_csv(r'out_lower.csv',index = False)

NameError: name 'df_out' is not defined